# Calculate the background photon rate

## IMPORT all needed packages

In [63]:
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
import pandas as pd
import re
from pathlib import Path
import math

## Defining a truncater
Truncates the number into how many digits are defined with digits

In [64]:
def truncate(number, digits) -> float:
    stepper = pow(10.0, digits)
    return math.trunc(stepper * number) / stepper

## Define needed parameters for all files

All changes needed should be in the next box.

### cooldown
should be the number of the cooldown. This will in the next section define a path to your data folder. Your main data folder per cooldown should look like LB077 where the last 3 numbers are the cooldown (e.g. 077).

### pv
This is the date of the reduction. Or whatever you want to use to separate between different tries. This is added to the excel filename, so if you don't want to overwrite your old file, change this. If you are doing several versions during one day, add a modifier. This is a string, so it doesn't need to be a number. So adding v1 for example will work.

### fileStart
This is the number of the first file the code considers. If you know you have files that are not good in your set, you can define the range starting from this. This code isn't sophisticated enough to handle a gap between files, for this you need to change the code, include the bad files, or do the analysis in 2 or more parts and combine the results by hand.

### fileEnd
This is the number of the last file the code considers. If you know you have files that are not good in your set, you can define the range ending to this.

## MAKE ALL CHANGES HERE

In [65]:
cooldown = '077'
cd='LB'+cooldown
pv = '20191206'
fileStart= 5
fileEnd =30
#Path and name for the result excel, make sure you have any of the folders you define here, the code won't make them
df_name='Cooldown_results/Cooldown_77/FORCAST_bg_%s_%s.xlsx' %(cd, pv)

## Add the paths to the files into a list

Prints the files at the end for checkup. If you don't like this feature, comment it out with #. But in general it's good to check that you get the files you are expecting to get (and also the amount of files), especially if you have made changes to the code.

In [66]:
b='B/'
r='R/'
blue='LB'+cooldown+'/'+b
red='LB'+cooldown+'/'+r

file_list = []


rootdirB = Path(blue)
rootdirR = Path(red)
# Return a list of regular files only, not directories
file_listB = [f for f in rootdirB.glob('**/*') if f.is_file()]
file_listR = [f for f in rootdirR.glob('**/*') if f.is_file()]

for line in file_listB:
    file_list.append(line)
for line in file_listR:
    file_list.append(line)
    

mm_files = []

for row in file_list:
    if row.parts[2].startswith('.'):
        continue
    else:
        numbers = re.findall('\d+', row.parts[2])
        #print(row.parts[2])
        if(int(numbers[1]) > fileStart and int(numbers[1]) < fileEnd):
            file = fits.open(row)
            if (file[0].header['APERTURE'] == 'open'):
            #print(row.parts[1])
                mm_files.append(row)

print(mm_files)

[PosixPath('LB077/B/bLB077_0023.fits'), PosixPath('LB077/B/bLB077_0022.fits'), PosixPath('LB077/B/bLB077_0009.fits'), PosixPath('LB077/B/bLB077_0025.fits'), PosixPath('LB077/B/bLB077_0013.fits'), PosixPath('LB077/B/bLB077_0029.fits'), PosixPath('LB077/B/bLB077_0028.fits'), PosixPath('LB077/B/bLB077_0012.fits'), PosixPath('LB077/B/bLB077_0024.fits'), PosixPath('LB077/B/bLB077_0008.fits'), PosixPath('LB077/B/bLB077_0027.fits'), PosixPath('LB077/B/bLB077_0011.fits'), PosixPath('LB077/B/bLB077_0007.fits'), PosixPath('LB077/B/bLB077_0006.fits'), PosixPath('LB077/B/bLB077_0010.fits'), PosixPath('LB077/B/bLB077_0026.fits'), PosixPath('LB077/B/bLB077_0021.fits'), PosixPath('LB077/R/rLB077_0026.fits'), PosixPath('LB077/R/rLB077_0027.fits'), PosixPath('LB077/R/rLB077_0020.fits'), PosixPath('LB077/R/rLB077_0016.fits'), PosixPath('LB077/R/rLB077_0017.fits'), PosixPath('LB077/R/rLB077_0021.fits'), PosixPath('LB077/R/rLB077_0018.fits'), PosixPath('LB077/R/rLB077_0022.fits'), PosixPath('LB077/R/rLB07

## Open and extract information on each file and save it to dataframe

First we create a data frame called forcast_bg with all the columns we will need.

After, we loop over the files in the mm_files list created in the previous part: as in we do this for each file:
* We open the file
* We define the data and the header part of the fits file
* We separate the actual filename from the path for the purpose of the table
* Several header information parameters are defined, these are used in the calculations and also printed into the data frame
* We define the PA offset depending on channel and whether the data was taken in low or high cap. These are predetermined numbers and not needed to be changed (1222 for SWC low-cap, 1280 for SWC hi-cap, 1370 for LWC low-cap, and 1000 for LWC hi-cap)
* Finally, we calculate the mean, median and standard deviation from the sigma clipped image (anything over 2.0 sigma has been excluded)
* The background is calculated: the PA offset is deducted from the median and then multiplied by the framerate and eperadu (electrons per count)
* For our purposes the background is also divided by 10ˆ6 and truncated to one digit after a point, this is our photoelectron background rate (million electrons per second)
* Finally we save the wanted information into a dataframe and write it out to excel

In [67]:
forcast_bg = pd.DataFrame(columns=['filename','wavelength','ilowcap','aperture','dichroic','instcfgs','framerate','detchan','detbias','eperadu','raw_siglevel','pa_offset','full_pe_rate','pe_rate'])

In [68]:
for row in mm_files:
    filename = row
    
    file = fits.open(filename)
    file_map=fits.PrimaryHDU(data=file[0].data[0])
    file_map.header = file[0].header
    
    #fn= filename.parts[1]+'/'+filename.parts[2]
    fn= filename.parts[2]
    
    wavelength = file[0].header['WAVELNTH']
    dichroic = file[0].header['DICHROIC']
    aperture = file[0].header['APERTURE']
    instcfgs = file[0].header['INSTCFGS']
    
    framerate = float(file[0].header['FRMRATE'])
    #print(framerate)
    e_per_adu = float(file[0].header['EPERADU'])
    #print(e_per_adu)
    cap = file[0].header['ILOWCAP']
    #print(cap)
    chan = file[0].header['DETCHAN']
    #print(chan)
    detbias = file[0].header['DETBIAS']
    
    PA_offset = 0
    
    #1222 for SWC low-cap, 1280 for SWC hi-cap, 1370 for LWC low-cap, and 1000 for LWC hi-cap
    if cap == True:
        if chan == 'SW':
            PA_offset = 1222
        if chan == 'LW':
            PA_offset = 1370
    else:
        if chan == 'SW':
            PA_offset = 1280
        if chan == 'LW':
            PA_offset = 1000
 
    image=file_map.data
    
    mean, median, std = sigma_clipped_stats(image, sigma=2.)   
    
    background = (median - PA_offset)*framerate*e_per_adu
    bg_mod = truncate(background/(10**6),1)

    bg = pd.Series([fn,wavelength,cap, aperture, dichroic,instcfgs,framerate, chan, detbias, e_per_adu, median,PA_offset, background, bg_mod],index=['filename','wavelength','ilowcap','aperture','dichroic','instcfgs','framerate','detchan','detbias','eperadu','raw_siglevel','pa_offset','full_pe_rate','pe_rate'])
    #print(bs)
    forcast_bg=forcast_bg.append(bg, ignore_index=True)
    

In [69]:
forcast_bg=forcast_bg.sort_values(by=['wavelength'])
forcast_bg = forcast_bg.reset_index()
del forcast_bg['index']
display(forcast_bg)

,filename,wavelength,ilowcap,aperture,dichroic,instcfgs,framerate,detchan,detbias,eperadu,raw_siglevel,pa_offset,full_pe_rate,pe_rate
0,bLB077_0021.fits,5.6,True,open,Dichroic,F056_Barr2,30.5176,SW,1.745,136.0,4486.000000,1222,1.354688e+07,13.5
1,bLB077_0006.fits,5.6,True,open,Mirror (swc),F056,119.6770,SW,1.739,136.0,4965.884766,1222,6.093574e+07,60.9
2,bLB077_0022.fits,6.4,True,open,Dichroic,F064_Barr2,244.1410,SW,1.751,136.0,4629.842285,1222,1.131512e+08,113.1
3,bLB077_0007.fits,6.4,True,open,Mirror (swc),F064,277.4330,SW,1.753,136.0,5005.590332,1222,1.427582e+08,142.7
4,bLB077_0029.fits,6.5,False,open,Mirror (swc),G063,254.3130,SW,1.738,1294.0,5551.971191,1280,1.405825e+09,1405.8
5,bLB077_0023.fits,7.7,False,open,Dichroic,F077_Barr2,92.4775,SW,1.737,1294.0,6508.733398,1280,6.257010e+08,625.7
6,bLB077_0008.fits,7.7,False,open,Mirror (swc),F077,127.1570,SW,1.739,1294.0,6825.988281,1280,9.125433e+08,912.5
7,bLB077_0009.fits,8.8,False,open,Mirror (swc),F088,145.3220,SW,1.739,1294.0,6433.518066,1280,9.691019e+08,969.1
8,bLB077_0024.fits,8.8,False,open,Dichroic,F088_Barr2,127.1570,SW,1.744,1294.0,6745.799805,1280,8.993490e+08,899.3
9,bLB077_0011.fits,11.1,False,open,Mirror (swc),F111,359.0300,SW,1.709,1294.0,6288.088379,1280,2.326682e+09,2326.6


In [70]:
cd='LB'+cooldown

writer = pd.ExcelWriter(df_name)
forcast_bg.to_excel(writer,'BG_rate',index=False)
writer.save()